In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext tensorboard
import datetime

!rm -rf ./logs/RNN

#tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    #we need to transpose data sets to be compatable with our weight vectors
    data_y = data[:, 0]
    data_y-=1 #binaryization of the categorical data

    data_x = data[:, 1:]
    data_x = convert_3D(normalize(data_x))
    return data_x, data_y

In [5]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
training_data_x = convert_3D(normalize(training_data_x)) 
test_x = convert_3D(normalize(test_x)) 
_, training_data_y = features_and_labels("../archive/exoTrain.csv")
_, test_y = features_and_labels("../archive/exoTest.csv")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

FileNotFoundError: [Errno 2] No such file or directory: '../archive/exoTrain.csv'

In [ ]:
RNNmodel = Sequential([
    SimpleRNN(32, input_shape=(training_data_x.shape[1:]), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

RNNmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/RNN/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasRNN/training.csv',separator=",", append=False)

RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = RNNmodel
model.save('keras-models/KerasRNN')

Epoch 1/3
112/112 [==============================] - 574s 5s/step - loss: 0.6933 - accuracy: 0.8194 - val_loss: 0.6932 - val_accuracy: 0.9594
Epoch 2/3
112/112 [==============================] - 539s 5s/step - loss: 0.6880 - accuracy: 0.9663 - val_loss: 0.6871 - val_accuracy: 0.9712
Epoch 3/3
112/112 [==============================] - ETA: 0s - loss: 0.6818 - accuracy: 0.9775

In [ ]:
%tensorboard --logdir logs/RNN/fit